# New York AirBnB

__Import statements__

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import (
    train_test_split,
    cross_validate
)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn import datasets, ensemble
from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn import set_config
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingRegressor

from sklearn.feature_selection import SequentialFeatureSelector

### Importing the dataset

In [ ]:
data = pd.read_csv("../data/raw/airbnb.csv")

/Users/valliakella/opt/miniconda3/envs/abnb/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,48,2019-11-04,0.33,3,322,0,NaN
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,1,408,2021-06-29,4.91,1,220,38,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2016-06-05,0.53,2,365,0,NaN
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,5,2,2021-08-08,0.02,1,91,1,NaN
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,2,505,2021-10-20,3.70,1,218,31,NaN


In [ ]:
# Drop license, id, last_review
# data.drop(columns=["id", "license", "last_review"], inplace=True)

In [ ]:
data.dropna(subset=["reviews_per_month"], inplace=True)

In [ ]:
data["year"] = pd.DatetimeIndex(data["last_review"]).year
data["month"] = pd.DatetimeIndex(data["last_review"]).month
data["date"] = pd.DatetimeIndex(data["last_review"]).day

In [ ]:
# Splitting the data

In [ ]:
train_big_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
train_data, val_data = train_test_split(train_big_data, test_size=0.25, random_state=123)

In [ ]:
## Some Stupidity

# def find_no_of_listings(data):
#     temp_df = pd.DataFrame(data["host_id"].value_counts()).reset_index()

#     temp_df.rename(columns={
#         "index": "host_id",
#         "host_id": "number_of_listings"
#     }, inplace=True)

#     data = pd.merge(data, temp_df, on="host_id")

#     data["number_of_listings"] = pd.cut(
#         data.number_of_listings,
#         bins=[0, 1, 10, np.inf],
#         labels=["1", "2-10", ">10"]
#     )

#     return data

In [ ]:
# find_no_of_listings(train_data)["number_of_listings"].value_counts()

In [ ]:
# find_no_of_listings(test_data)["number_of_listings"].value_counts()

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [ ]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license,year,month,date
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,...,48,2019-11-04,0.33,3,322,0,NaN,2019,11,4
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,...,408,2021-06-29,4.91,1,220,38,NaN,2021,6,29
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,...,50,2016-06-05,0.53,2,365,0,NaN,2016,6,5
3,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Brooklyn,Sunset Park,40.66265,-73.99454,Entire home/apt,275,...,2,2021-08-08,0.02,1,91,1,NaN,2021,8,8
4,5178,Large Furnished Room Near B'way,8967,Shunichi,Manhattan,Midtown,40.76457,-73.98317,Private room,68,...,505,2021-10-20,3.70,1,218,31,NaN,2021,10,20


In [ ]:
# find_no_of_listings(data)[["number_of_listings"]].value_counts()

In [ ]:
# pd.DataFrame(data["neighbourhood"].value_counts()).to_csv("temp.csv")

In [ ]:
# data["number_of_listings"] =

In [66]:
numeric_features = [
    "latitude",
    "longitude",
    "price",
    "minimum_nights",
    "calculated_host_listings_count",
    "availability_365",
    "year",
    "date",
    
]

categorical_features = [
    "neighbourhood_group",
    "neighbourhood",
    "room_type",
    "month"
]

text_features = [
    "name"
]

drop_features = [
    "id",
    "license",
    "host_id",
    "host_name",
    "number_of_reviews",
    "number_of_reviews_ltm",
     "last_review"
]

passthrough_features = [ 
   
    
]

target_column = "reviews_per_month"

assert (
    len(numeric_features) +
    len(categorical_features) +
    len(text_features) +
    len(drop_features) +
    len(passthrough_features) +
    len([target_column])
) == len(train_data.columns)

In [67]:
X_train, y_train = train_data.drop(columns=[target_column]), train_data[target_column]
X_big_train, y_big_train = train_big_data.drop(columns=[target_column]), train_big_data[target_column]

In [68]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17100 entries, 19360 to 13967
Data columns (total 20 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              17100 non-null  int64  
 1   name                            17098 non-null  object 
 2   host_id                         17100 non-null  int64  
 3   host_name                       17094 non-null  object 
 4   neighbourhood_group             17100 non-null  object 
 5   neighbourhood                   17100 non-null  object 
 6   latitude                        17100 non-null  float64
 7   longitude                       17100 non-null  float64
 8   room_type                       17100 non-null  object 
 9   price                           17100 non-null  int64  
 10  minimum_nights                  17100 non-null  int64  
 11  number_of_reviews               17100 non-null  int64  
 12  last_review                 

In [69]:
function_transformer = FunctionTransformer(
    np.reshape, kw_args={"newshape": -1}
)

pipe_text_feats = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    function_transformer,
    CountVectorizer(stop_words="english", max_features=30)
)

column_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), categorical_features),
    (pipe_text_feats, text_features),
    ("drop", drop_features),
    ("passthrough", passthrough_features)
)

In [70]:
column_transformer.fit(X_big_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 ['latitude', 'longitude', 'price',
                                  'minimum_nights',
                                  'calculated_host_listings_count',
                                  'availability_365', 'year', 'date']),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 ['neighbourhood_group', 'neighbourhood',
                                  'room_type', 'month']),
                                ('pipeline',
                                 Pipeline(steps=[('simple...
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('functiontransformer',
                                                  FunctionTransformer(func=<function

In [71]:
new_columns = (
    numeric_features +
    column_transformer.named_transformers_["onehotencoder"].get_feature_names_out().tolist() +
    column_transformer.named_transformers_["pipeline"].named_steps["countvectorizer"].get_feature_names_out().tolist() +
    passthrough_features
)

In [72]:
pd.DataFrame(column_transformer.transform(X_big_train), columns=new_columns)

,latitude,longitude,price,minimum_nights,calculated_host_listings_count,availability_365,year,date,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,...,new,nyc,park,private,room,spacious,studio,sunny,village,williamsburg
0,1.524150,-0.273302,-0.296471,0.401279,-0.261303,-0.905926,-1.927383,0.380548,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.171043,-0.358917,0.423702,0.401279,-0.261303,0.387799,-1.389311,-0.844930,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,-0.684770,-0.032217,-0.412499,0.591294,0.591129,1.515291,0.224905,1.197533,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.791457,-0.019375,-0.220453,-0.662806,-0.261303,-0.905926,0.224905,0.074178,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.514169,0.809341,-0.076419,-0.548797,-0.227206,-0.905926,0.224905,0.482671,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22795,2.177763,0.205754,-0.300472,0.401279,-0.227206,1.088867,0.762977,0.891164,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
22796,-0.284570,-0.043309,-0.316476,-0.358782,-0.261303,-0.905926,0.224905,0.993287,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
22797,-0.951333,0.147380,-0.416500,-0.662806,-0.261303,-0.667418,0.762977,1.503903,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22798,-0.592539,0.075969,-0.356486,0.401279,-0.261303,-0.905926,-0.313167,0.482671,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# Credits to Varada K.

def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [74]:
from sklearn.metrics import make_scorer


def mape(true, pred): # adapted from notes
    return 100.0 * np.mean(np.abs((pred - true) / true))


# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer,
}

### Baseline Regression

In [75]:
results = {}

In [76]:
models = {
    "Dummy": make_pipeline(
        column_transformer, DummyRegressor()
    ),
    "Ridge": make_pipeline(
        column_transformer, Ridge()
    ),
    "Random_Forest_reg": make_pipeline(
         column_transformer, RandomForestRegressor(random_state=123)
     ),
    "XGBoost_reg": make_pipeline(
        column_transformer, XGBRegressor(verbosity=0)
    ),
    "lgbm_reg": make_pipeline(
        column_transformer, LGBMRegressor()
    ),
     "catBoost_reg": make_pipeline(
         column_transformer, CatBoostRegressor(verbose=0)
     )
}

    
for model_name, model in models.items():
    results[model_name] = mean_std_cross_val_scores(
        model, 
        X_big_train, 
        y_big_train, 
        cv=10, 
        return_train_score=True, 
        scoring=scoring_metrics
 )
    

In [77]:
pd.DataFrame(results)

,Dummy,Ridge,Random_Forest_reg,XGBoost_reg,lgbm_reg,catBoost_reg
fit_time,0.141 (+/- 0.017),0.217 (+/- 0.014),27.024 (+/- 0.828),6.317 (+/- 0.578),0.393 (+/- 0.038),5.369 (+/- 0.487)
score_time,0.018 (+/- 0.002),0.019 (+/- 0.001),0.097 (+/- 0.031),0.028 (+/- 0.006),0.024 (+/- 0.004),0.146 (+/- 0.030)
test_neg RMSE,-4.283 (+/- 0.646),-4.009 (+/- 0.655),-3.951 (+/- 0.570),-3.926 (+/- 0.530),-3.742 (+/- 0.584),-3.832 (+/- 0.598)
train_neg RMSE,-4.326 (+/- 0.076),-3.986 (+/- 0.076),-1.468 (+/- 0.023),-2.190 (+/- 0.050),-3.041 (+/- 0.054),-2.819 (+/- 0.060)
test_r2,-0.000 (+/- 0.000),0.126 (+/- 0.025),0.145 (+/- 0.064),0.155 (+/- 0.047),0.237 (+/- 0.034),0.199 (+/- 0.040)
train_r2,0.000 (+/- 0.000),0.151 (+/- 0.003),0.885 (+/- 0.003),0.744 (+/- 0.006),0.506 (+/- 0.009),0.575 (+/- 0.011)
test_mape,-1432.611 (+/- 68.500),-789.402 (+/- 40.564),-215.711 (+/- 7.807),-329.392 (+/- 41.685),-254.364 (+/- 11.551),-297.988 (+/- 17.139)
train_mape,-1432.549 (+/- 10.522),-781.856 (+/- 8.078),-79.668 (+/- 0.530),-243.846 (+/- 5.928),-237.478 (+/- 3.058),-267.666 (+/- 2.938)


### Hyperparameter Tuning

In [ ]:
#Hyperparameter tune the Ridge

In [66]:
param_grid = {'ridge__alpha': 10.0 ** np.arange(-2, 6, 1)}

In [67]:
pipe_lr = make_pipeline(column_transformer, Ridge())

random_search = RandomizedSearchCV(
    pipe_lr,
    param_grid,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit='r2'
)

In [68]:
random_search.fit(X_train, y_train)

/Users/valliakella/opt/miniconda3/envs/abnb/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('standardscaler',
                                                                               StandardScaler(),
                                                                               ['latitude',
                                                                                'longitude',
                                                                                'price',
                                                                                'minimum_nights',
                                                                                'calculated_host_listings_count',
                                                                                'availability_365',
                                                                                'year',
                                                                 

In [71]:
pd.DataFrame(random_search.cv_results_)[[
        "mean_fit_time", 
        "mean_score_time", 
        "param_ridge__alpha", 
        "mean_train_neg RMSE", 
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

,mean_fit_time,mean_score_time,param_ridge__alpha,mean_train_neg RMSE,std_train_neg RMSE,mean_test_mape,mean_train_mape,mean_test_r2,mean_train_r2
5,0.319665,0.040711,1000.0,-4.117188,0.117993,-792.836021,-786.484132,0.102000,0.105800
4,0.310543,0.048999,100.0,-4.087348,0.118544,-859.248289,-847.112298,0.101773,0.118730
3,0.334640,0.044049,10.0,-4.058977,0.122061,-889.386003,-872.124774,0.094758,0.130964
6,0.310296,0.035848,10000.0,-4.158164,0.118153,-672.030307,-670.253613,0.089017,0.087902
2,0.370175,0.046060,1.0,-4.031845,0.125979,-897.829242,-876.300095,0.088637,0.142587
1,0.424338,0.051780,0.1,-4.024586,0.127083,-900.140454,-876.667551,0.086349,0.145682
0,0.476041,0.058262,0.01,-4.024353,0.127119,-900.682890,-876.838280,0.085969,0.145781
7,0.273776,0.040972,100000.0,-4.283499,0.116283,-1201.607871,-1201.521823,0.032823,0.032028


In [72]:
pipe_lr_final = make_pipeline(column_transformer, Ridge(alpha=1000))
pipe_lr_final.fit(X_train, y_train)

import eli5

eli5.explain_weights(pipe_lr_final.named_steps["ridge"], feature_names=new_columns)

Weight?,Feature
+1.690,<BIAS>
+0.758,year
+0.516,month
+0.319,hotel
+0.276,king
+0.248,neighbourhood_Financial District
+0.242,cozy
+0.198,view
+0.197,bed
+0.193,jfk


In [75]:
data={
    "Importance": pipe_lr_final.named_steps["ridge"].coef_,
}
pd.DataFrame(data=data, index=new_columns,).sort_values(
    by="Importance", ascending=False
)[:10]


,Importance
year,0.757801
month,0.516439
hotel,0.318748
king,0.275677
neighbourhood_Financial District,0.248030
cozy,0.242420
view,0.197933
bed,0.196688
jfk,0.192802
queen,0.174597
